In [22]:
import sktime
import numpy as np
import pandas as pd

In [23]:
df = pd.read_csv('../data/intermediate_data/preprocess_data/hackathon_df_preprocessed.csv', index_col=0, parse_dates=True)
df

,ret_eom,permno,shrcd,exchcd,mspread,year,month,size_port,rf,stock_exret,...,betadown_252d,bidaskhl_21d,corr_1260d,betabab_1260d,rmax5_rvol_21d,age,qmj,qmj_prof,qmj_growth,qmj_safety
date,,,,,,,,,,,,,,,,,,,,,
2000-01-31,2000-01-31,10104,11,3,0.019720,2000,1,B,0.0041,-0.112577,...,1.437330,0.016634,0.473872,1.941648,2.058353,180,0.828930,1.717670,-0.603531,0.608215
2000-01-31,2000-01-31,10107,11,3,0.011369,2000,1,B,0.0041,-0.165770,...,1.466883,0.003855,0.563178,1.228124,1.899772,180,1.190962,1.682187,-0.583515,1.336684
2000-01-31,2000-01-31,10119,11,1,0.003485,2000,1,B,0.0041,-0.036462,...,0.217867,0.004411,0.289368,0.457370,0.715285,228,0.785417,-0.381167,1.689625,0.331900
2000-01-31,2000-01-31,10138,11,3,0.016904,2000,1,B,0.0041,0.048353,...,1.101137,0.011532,0.543539,1.523013,1.222404,180,1.668739,1.579503,1.168335,1.312634
2000-01-31,2000-01-31,10145,11,1,0.008796,2000,1,B,0.0041,-0.172031,...,0.395308,0.007589,0.533192,1.091895,1.191058,888,1.431156,1.038128,1.163113,0.814517
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-29,2023-12-31,93330,11,1,0.004891,2023,12,B,0.0043,-0.022199,...,0.946309,0.003779,0.720817,1.241060,1.464466,887,0.683460,0.169549,0.627055,0.745462
2023-12-29,2023-12-31,93356,11,3,0.004867,2023,12,B,0.0043,0.120845,...,1.481037,0.007985,0.642513,1.388323,1.578243,227,1.679546,1.550166,0.993086,1.619852
2023-12-29,2023-12-31,93369,11,3,0.018935,2023,12,B,0.0043,-0.005245,...,1.239248,0.012130,0.454100,1.339266,1.407091,191,-0.039003,-0.435984,1.397522,-0.570584


In [24]:
def convert_to_panel_with_multiindex(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convert a DataFrame with 'stock_ticker' and 'date' (as index) to a hierarchical panel format with a MultiIndex.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame with 'stock_ticker' column and the 'date' as the index.
    
    Returns:
    pd.DataFrame: A DataFrame with a MultiIndex (stock_ticker, date) sorted by stock_ticker and date.
    """
    
    # Ensure the index (date) is in datetime format
    df.index = pd.to_datetime(df.index, errors='coerce')
    
    # Set the MultiIndex with 'stock_ticker' as the first level and 'date' (currently the index) as the second level
    df_multiindex = df.set_index(['stock_ticker', df.index], append=False)
    
    # Sort the index by both stock_ticker and date (date chronologically)
    df_multiindex = df_multiindex.sort_index(level=[0, 1])
    
    return df_multiindex

In [25]:
df_multiindex = convert_to_panel_with_multiindex(df)
df_multiindex

ret_eom  permno  shrcd  exchcd   mspread  year  \
stock_ticker date                                                            
A            2000-12-29  2000-12-31   87432     11       1  0.016473  2000   
             2001-01-31  2001-01-31   87432     11       1  0.011525  2001   
             2001-02-28  2001-02-28   87432     11       1  0.009882  2001   
             2001-03-30  2001-03-31   87432     11       1  0.015782  2001   
             2001-04-30  2001-04-30   87432     11       1  0.018775  2001   
...                             ...     ...    ...     ...       ...   ...   
ZWS          2023-08-31  2023-08-31   13305     11       1  0.007623  2023   
             2023-09-29  2023-09-30   13305     11       1  0.004933  2023   
             2023-10-31  2023-10-31   13305     11       1  0.005457  2023   
             2023-11-30  2023-11-30   13305     11       1  0.004726  2023   
             2023-12-29  2023-12-31   13305     11       1  0.005622  2023   

                         month size_port      rf  stock_exret  ...  \
stock_ticker date                                              ...   
A            2000-12-29     12         B  0.0050     0.044102  ...   
             2001-01-31      1         B  0.0054    -0.009053  ...   
             2001-02-28      2         B  0.0038    -0.343855  ...   
             2001-03-30      3         B  0.0042    -0.150589  ...   
             2001-04-30      4         B  0.0039     0.265544  ...   
...                        ...       ...     ...          ...  ...   
ZWS          2023-08-31      8         B  0.0045    -0.029139  ...   
             2023-09-29      9         B  0.0043    -0.058318  ...   
             2023-10-31     10         B  0.0047    -0.060375  ...   
             2023-11-30     11         B  0.0044     0.111246  ...   
             2023-12-29     12         B  0.0043    -0.005319  ...   

                        betadown_252d bidaskhl_21d  corr_1260d  betabab_1260d  \
stock_ticker date                                                               
A            2000-12-29      2.234551     0.013440         NaN            NaN   
             2001-01-31      2.160193     0.016473         NaN            NaN   
             2001-02-28      2.209745     0.011525         NaN            NaN   
             2001-03-30      2.444064     0.009882         NaN            NaN   
             2001-04-30      2.223938     0.015782         NaN            NaN   
...                               ...          ...         ...            ...   
ZWS          2023-08-31      1.247594     0.004564    0.671186       1.299755   
             2023-09-29      1.145259     0.007623    0.672056       1.302454   
             2023-10-31      1.094390     0.004933    0.672301       1.345981   
             2023-11-30      1.060510     0.005457    0.665731       1.350127   
             2023-12-29      1.132624     0.004726    0.667397       1.472506   

                         rmax5_rvol_21d  age       qmj  qmj_prof  qmj_growth  \
stock_ticker date                                                              
A            2000-12-29        1.267846   35       NaN  1.119352         NaN   
             2001-01-31        0.972802   36       NaN  1.115908         NaN   
             2001-02-28        1.010576   37       NaN  1.116300         NaN   
             2001-03-30        0.741048   38       NaN  1.373031         NaN   
             2001-04-30        1.251023   39       NaN  1.374396         NaN   
...                                 ...  ...       ...       ...         ...   
ZWS          2023-08-31        1.611774  163 -0.610793  0.352985   -0.935608   
             2023-09-29        0.434724  164 -0.602658  0.339540   -0.934711   
             2023-10-31        0.408287  165 -0.601575  0.339696   -0.939739   
             2023-11-30        0.865284  166 -0.285945  0.556308   -0.695799   
             2023-12-29        1.611720  167 -0.293426  0.540943   -0.702661   

       

In [26]:
def shift_columns(df: pd.DataFrame, shift_dict: dict) -> pd.DataFrame:
    """
    Shift the values of specific columns in a MultiIndex DataFrame according to the given periods.
    
    Parameters:
    ----------
    df : pd.DataFrame
        A MultiIndex DataFrame where the index is (stock_ticker, date).
    shift_dict : dict
        A dictionary where the key is the column name to shift and the value is the period to shift.
        For example, {'column1': 1, 'column2': -2} will shift 'column1' down by 1 period and 'column2' up by 2 periods.
        
    Returns:
    -------
    pd.DataFrame
        A DataFrame with the specified columns shifted by the given periods.
    """
    
    # We will create a copy of the DataFrame to avoid modifying the original one
    df_shifted = df.copy()
    
    # Loop through the dictionary to shift each column
    for col, period in shift_dict.items():
        if col in df_shifted.columns:
            df_shifted[col] = df_shifted.groupby(level=0)[col].shift(period)
        else:
            raise KeyError(f"Column '{col}' not found in the DataFrame.")
    # drop rows where stock_exret is NaN
    df_shifted = df_shifted.dropna(subset=['stock_exret'])
    return df_shifted

In [27]:
shift_dict = {'stock_exret': 1, 'eps_meanest': 1, 'eps_medest': 1, 'eps_stdevest': 1}
df_shifted = shift_columns(df_multiindex, shift_dict)
df_shifted

ret_eom  permno  shrcd  exchcd   mspread  year  \
stock_ticker date                                                            
A            2001-01-31  2001-01-31   87432     11       1  0.011525  2001   
             2001-02-28  2001-02-28   87432     11       1  0.009882  2001   
             2001-03-30  2001-03-31   87432     11       1  0.015782  2001   
             2001-04-30  2001-04-30   87432     11       1  0.018775  2001   
             2001-05-31  2001-05-31   87432     11       1  0.011937  2001   
...                             ...     ...    ...     ...       ...   ...   
ZWS          2023-08-31  2023-08-31   13305     11       1  0.007623  2023   
             2023-09-29  2023-09-30   13305     11       1  0.004933  2023   
             2023-10-31  2023-10-31   13305     11       1  0.005457  2023   
             2023-11-30  2023-11-30   13305     11       1  0.004726  2023   
             2023-12-29  2023-12-31   13305     11       1  0.005622  2023   

                         month size_port      rf  stock_exret  ...  \
stock_ticker date                                              ...   
A            2001-01-31      1         B  0.0054     0.044102  ...   
             2001-02-28      2         B  0.0038    -0.009053  ...   
             2001-03-30      3         B  0.0042    -0.343855  ...   
             2001-04-30      4         B  0.0039    -0.150589  ...   
             2001-05-31      5         B  0.0032     0.265544  ...   
...                        ...       ...     ...          ...  ...   
ZWS          2023-08-31      8         B  0.0045     0.127519  ...   
             2023-09-29      9         B  0.0043    -0.029139  ...   
             2023-10-31     10         B  0.0047    -0.058318  ...   
             2023-11-30     11         B  0.0044    -0.060375  ...   
             2023-12-29     12         B  0.0043     0.111246  ...   

                        betadown_252d bidaskhl_21d  corr_1260d  betabab_1260d  \
stock_ticker date                                                               
A            2001-01-31      2.160193     0.016473         NaN            NaN   
             2001-02-28      2.209745     0.011525         NaN            NaN   
             2001-03-30      2.444064     0.009882         NaN            NaN   
             2001-04-30      2.223938     0.015782         NaN            NaN   
             2001-05-31      2.383538     0.018775         NaN            NaN   
...                               ...          ...         ...            ...   
ZWS          2023-08-31      1.247594     0.004564    0.671186       1.299755   
             2023-09-29      1.145259     0.007623    0.672056       1.302454   
             2023-10-31      1.094390     0.004933    0.672301       1.345981   
             2023-11-30      1.060510     0.005457    0.665731       1.350127   
             2023-12-29      1.132624     0.004726    0.667397       1.472506   

                         rmax5_rvol_21d  age       qmj  qmj_prof  qmj_growth  \
stock_ticker date                                                              
A            2001-01-31        0.972802   36       NaN  1.115908         NaN   
             2001-02-28        1.010576   37       NaN  1.116300         NaN   
             2001-03-30        0.741048   38       NaN  1.373031         NaN   
             2001-04-30        1.251023   39       NaN  1.374396         NaN   
             2001-05-31        1.484360   40       NaN  1.369927         NaN   
...                                 ...  ...       ...       ...         ...   
ZWS          2023-08-31        1.611774  163 -0.610793  0.352985   -0.935608   
             2023-09-29        0.434724  164 -0.602658  0.339540   -0.934711   
             2023-10-31        0.408287  165 -0.601575  0.339696   -0.939739   
             2023-11-30        0.865284  166 -0.285945  0.556308   -0.695799   
             2023-12-29        1.611720  167 -0.293426  0.540943   -0.702661   

       

In [28]:
def extract_target(df: pd.DataFrame, target_col: str, drop_cols: list = None) -> (pd.DataFrame, pd.Series):
    """
    Extract the target column (y) and the feature columns (X) from a MultiIndex DataFrame.
    
    Parameters:
    ----------
    df : pd.DataFrame
        A MultiIndex DataFrame where the index is (stock_ticker, date).
    target_col : str
        The name of the column to be used as the target (y) for the model.
    drop_cols : list, optional
        A list of column names to drop from the feature set (X).
    
    Returns:
    -------
    X : pd.DataFrame
        The feature DataFrame (X) with specified columns dropped.
    y : pd.Series
        The target Series (y) corresponding to the target column.
    """
    
    # Check if the target column exists in the DataFrame
    if target_col not in df.columns:
        raise KeyError(f"Target column '{target_col}' not found in the DataFrame.")
    
    # Extract the target column (y)
    y = df[target_col]
    
    # Create the feature set (X) by dropping the target column and any specified columns
    X = df.drop(columns=[target_col])
    
    if drop_cols:
        X = X.drop(columns=drop_cols)
    
    return pd.DataFrame(X), pd.DataFrame(y)

In [29]:
df.head(1)

,ret_eom,permno,shrcd,exchcd,mspread,year,month,size_port,rf,stock_exret,...,betadown_252d,bidaskhl_21d,corr_1260d,betabab_1260d,rmax5_rvol_21d,age,qmj,qmj_prof,qmj_growth,qmj_safety
date,,,,,,,,,,,,,,,,,,,,,
2000-01-31,2000-01-31,10104,11,3,0.01972,2000,1,B,0.0041,-0.112577,...,1.43733,0.016634,0.473872,1.941648,2.058353,180,0.82893,1.71767,-0.603531,0.608215


In [30]:
target_col = 'stock_exret'
drop_cols = ['ret_eom', 'permno', 'shrcd', 'exchcd', 'year', 'month', 'size_port', 'cusip', 'comp_name']
X, y = extract_target(df_shifted, target_col, drop_cols)
X

mspread      rf  eps_medest  eps_meanest  \
stock_ticker date                                                    
A            2001-01-31  0.011525  0.0054        0.45         0.45   
             2001-02-28  0.009882  0.0038        0.45         0.45   
             2001-03-30  0.015782  0.0042        0.45         0.45   
             2001-04-30  0.018775  0.0039        0.35         0.34   
             2001-05-31  0.011937  0.0032        0.30         0.27   
...                           ...     ...         ...          ...   
ZWS          2023-08-31  0.007623  0.0045        0.22         0.22   
             2023-09-29  0.004933  0.0043        0.27         0.27   
             2023-10-31  0.005457  0.0047        0.27         0.27   
             2023-11-30  0.004726  0.0044        0.27         0.27   
             2023-12-29  0.005622  0.0043        0.23         0.23   

                         eps_stdevest  eps_actual        prc  market_equity  \
stock_ticker date                                                             
A            2001-01-31          0.04        0.51  54.750000   24802.571250   
             2001-02-28          0.03        0.51  54.549999   24894.764952   
             2001-03-30          0.03        0.11  36.000000   16429.176000   
             2001-04-30          0.04        0.11  30.730000   14036.541891   
             2001-05-31          0.06       -0.24  39.009998   17818.596933   
...                               ...         ...        ...            ...   
ZWS          2023-08-31          0.01        0.29  30.440001    5264.537212   
             2023-09-29          0.01        0.29  29.620001    5122.719905   
             2023-10-31          0.01        0.29  28.020000    4846.003039   
             2023-11-30          0.00        0.26  26.459999    4576.203922   
             2023-12-29          0.00        0.26  29.440001    5087.555932   

                         div12m_me  chcsho_12m  ...  betadown_252d  \
stock_ticker date                               ...                  
A            2001-01-31   0.000000    0.018011  ...       2.160193   
             2001-02-28   0.000000    0.009659  ...       2.209745   
             2001-03-30   0.000000    0.009659  ...       2.444064   
             2001-04-30   0.000000    0.010553  ...       2.223938   
             2001-05-31   0.000000    0.010553  ...       2.383538   
...                            ...         ...  ...            ...   
ZWS          2023-08-31   0.009389    0.372952  ...       1.247594   
             2023-09-29   0.009583   -0.026999  ...       1.145259   
             2023-10-31   0.010131   -0.026999  ...       1.094390   
             2023-11-30   0.010728   -0.027262  ...       1.060510   
             2023-12-29   0.009921   -0.028032  ...       1.132624   

                         bidaskhl_21d  corr_1260d  betabab_1260d  \
stock_ticker date                                                  
A            2001-01-31      0.016473         NaN            NaN   
             2001-02-28      0.011525         NaN            NaN   
             2001-03-30      0.009882         NaN            NaN   
             2001-04-30      0.015782         NaN            NaN   
             2001-05-31      0.018775         NaN            NaN   
...                               ...         ...            ...   
ZWS          2023-08-31      0.004564    0.671186       1.299755   
             2023-09-29      0.007623    0.672056       1.302454   
             2023-10-31      0.004933    0.672301       1.345981   
             2023-11-30      0.005457    0.665731       1.350127   
             2023-12-29      0.004726    0.667397       1.472506   

                         rmax5_rvol_21d  age       qmj  qmj_prof  qmj_growth  \
stock_ticker date                                                              
A            2001-01-31        0.972802   36       NaN  1.115908         NaN   
             2001-02-28        1.010576   37       

In [31]:
y

stock_exret
stock_ticker date                   
A            2001-01-31     0.044102
             2001-02-28    -0.009053
             2001-03-30    -0.343855
             2001-04-30    -0.150589
             2001-05-31     0.265544
...                              ...
ZWS          2023-08-31     0.127519
             2023-09-29    -0.029139
             2023-10-31    -0.058318
             2023-11-30    -0.060375
             2023-12-29     0.111246

[177001 rows x 1 columns]

In [32]:
type(y)

pandas.core.frame.DataFrame

In [33]:
from sktime.split import temporal_train_test_split
from sklearn.tree import ExtraTreeRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sktime.forecasting.compose import make_reduction
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.pytorchforecasting import PytorchForecastingDeepAR
from sktime.forecasting.arima import ARIMA



In [34]:
y_train, y_test, X_train, X_test = temporal_train_test_split(y, X, test_size=0.2)

In [35]:
y_train.shape, y_test.shape, X_train.shape, X_test.shape

((141015, 1), (35986, 1), (141015, 153), (35986, 153))

In [36]:
y_train

stock_exret
stock_ticker date                   
A            2001-01-31     0.044102
             2001-02-28    -0.009053
             2001-03-30    -0.343855
             2001-04-30    -0.150589
             2001-05-31     0.265544
...                              ...
ZWS          2023-02-28     0.030070
             2023-03-31     0.051952
             2023-04-28    -0.074904
             2023-05-31     0.005395
             2023-06-30     0.044196

[141015 rows x 1 columns]

In [37]:
y_test

stock_exret
stock_ticker date                   
A            2019-05-31    -0.023449
             2019-06-28    -0.147960
             2019-07-31     0.111846
             2019-08-30    -0.070147
             2019-09-30     0.022892
...                              ...
ZWS          2023-08-31     0.127519
             2023-09-29    -0.029139
             2023-10-31    -0.058318
             2023-11-30    -0.060375
             2023-12-29     0.111246

[35986 rows x 1 columns]

In [38]:
# count values by stock_ticker in the multi index (first level)
y_train.index.get_level_values(0).value_counts()

stock_ticker
MSFT    229
HD      229
PXD     229
JPM     229
K       229
       ... 
ENOV      8
OTTR      8
PRFT      8
OPEN      8
BMBL      8
Name: count, Length: 1320, dtype: int64

In [39]:
# count unique values by stock_ticker in the multi index (first level) that surpass a certain threshold, return int
y_train.index.get_level_values(0).value_counts().gt(100).sum()

580

In [40]:
regressor = HistGradientBoostingRegressor(max_iter=1)
model = make_reduction(estimator=regressor, window_length=5)

In [41]:
fh = ForecastingHorizon(values=[1], is_relative=True)
fh

ForecastingHorizon([1], dtype='int64', is_relative=True)

In [ ]:
# y_pred = model.fit_predict(y=y_train, X=X_train, fh=fh)

In [ ]:
# y_pred

In [22]:
X_train.index.get_level_values(1)

DatetimeIndex(['2001-01-31', '2001-02-28', '2001-03-30', '2001-04-30',
               '2001-05-31', '2001-06-29', '2001-07-31', '2001-08-31',
               '2001-09-28', '2001-10-31',
               ...
               '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-30',
               '2023-01-31', '2023-02-28', '2023-03-31', '2023-04-28',
               '2023-05-31', '2023-06-30'],
              dtype='datetime64[ns]', name='date', length=191354, freq=None)

In [23]:
X_train.index

MultiIndex([(  'A', '2001-01-31'),
            (  'A', '2001-02-28'),
            (  'A', '2001-03-30'),
            (  'A', '2001-04-30'),
            (  'A', '2001-05-31'),
            (  'A', '2001-06-29'),
            (  'A', '2001-07-31'),
            (  'A', '2001-08-31'),
            (  'A', '2001-09-28'),
            (  'A', '2001-10-31'),
            ...
            ('ZWS', '2022-09-30'),
            ('ZWS', '2022-10-31'),
            ('ZWS', '2022-11-30'),
            ('ZWS', '2022-12-30'),
            ('ZWS', '2023-01-31'),
            ('ZWS', '2023-02-28'),
            ('ZWS', '2023-03-31'),
            ('ZWS', '2023-04-28'),
            ('ZWS', '2023-05-31'),
            ('ZWS', '2023-06-30')],
           names=['stock_ticker', 'date'], length=191354)

In [24]:
X.index.get_level_values(1)

DatetimeIndex(['2001-01-31', '2001-02-28', '2001-03-30', '2001-04-30',
               '2001-05-31', '2001-06-29', '2001-07-31', '2001-08-31',
               '2001-09-28', '2001-10-31',
               ...
               '2023-03-31', '2023-04-28', '2023-05-31', '2023-06-30',
               '2023-07-31', '2023-08-31', '2023-09-29', '2023-10-31',
               '2023-11-30', '2023-12-29'],
              dtype='datetime64[ns]', name='date', length=240382, freq=None)

In [25]:
from sktime.utils._testing.hierarchical import _make_hierarchical

y = _make_hierarchical(min_timepoints=16, max_timepoints=16, all_positive=False)
y

c0
h0   h1   time                
h0_0 h1_0 2000-01-01  1.279812
          2000-01-02 -1.130340
          2000-01-03 -0.320275
          2000-01-04  1.428274
          2000-01-05 -0.365085
...                        ...
h0_1 h1_3 2000-01-12  0.813326
          2000-01-13  0.704041
          2000-01-14 -0.043425
          2000-01-15 -1.075457
          2000-01-16 -1.121487

[128 rows x 1 columns]

In [26]:
forecaster = ARIMA()
y_pred = forecaster.fit(y, fh=[1, 2]).predict()
y_pred

c0
h0   h1   time                
h0_0 h1_0 2000-01-17  0.068801
          2000-01-18  0.054722
     h1_1 2000-01-17 -0.284497
          2000-01-18  0.059070
     h1_2 2000-01-17  1.038206
          2000-01-18  0.255985
     h1_3 2000-01-17  0.933407
          2000-01-18  0.801944
h0_1 h1_0 2000-01-17  0.149513
          2000-01-18  0.129280
     h1_1 2000-01-17  0.273488
          2000-01-18  0.111481
     h1_2 2000-01-17  0.312186
          2000-01-18  0.135156
     h1_3 2000-01-17 -0.217722
          2000-01-18 -0.176520